**Numerical Examples From "Adversarial QA"**

In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset
adversarial_qa_dataset = load_dataset("adversarial_qa", "adversarialQA", split='train')

# saving to cloned directory
save_path = '/content/'
adversarial_qa_dataset.save_to_disk(save_path)

In [ ]:
# PRELIM ANALYSIS

# seeing the structure of the dataset
print(f"Total number of examples in the 'train' split:", len(adversarial_qa_dataset))
num_examples = int(len(adversarial_qa_dataset))
print(f"Number of examples iterating through:", num_examples)


# EXTRACTING NUMERIC ANSWERS

num_exs = []

# iterate through set number of examples and take out examples with number answers
for i in range(num_examples):
    # checkpoing for iterations
    if (i%1000)==0:
      print(f"Iterations complete (out of 30,000):", i)
    # extracting answers and splitting into words
    answer = adversarial_qa_dataset['answers'][i]
    answer_words = answer['text'][0].split()
    # print(answer)
    for word in answer_words:
      # print(word)
      if any(char.isdigit() for char in word):
        # print(answer)
        # adding answers to a list for json conversion
        example = {
        'question': adversarial_qa_dataset['question'][i],
        'context': adversarial_qa_dataset['context'][i],
        'answers': adversarial_qa_dataset['answers'][i],
      }
        num_exs.append(example)
        # don't have to look through each word, break if you've found one number
        break

print(f"Total number of numerical answers found:", len(num_exs))

Total number of examples in the 'train' split: 30000
Number of examples iterating through: 30000
Iterations complete (out of 30,000): 0
Iterations complete (out of 30,000): 1000
Iterations complete (out of 30,000): 2000
Iterations complete (out of 30,000): 3000
Iterations complete (out of 30,000): 4000
Iterations complete (out of 30,000): 5000
Iterations complete (out of 30,000): 6000
Iterations complete (out of 30,000): 7000
Iterations complete (out of 30,000): 8000
Iterations complete (out of 30,000): 9000
Iterations complete (out of 30,000): 10000
Iterations complete (out of 30,000): 11000
Iterations complete (out of 30,000): 12000
Iterations complete (out of 30,000): 13000
Iterations complete (out of 30,000): 14000
Iterations complete (out of 30,000): 15000
Iterations complete (out of 30,000): 16000
Iterations complete (out of 30,000): 17000
Iterations complete (out of 30,000): 18000
Iterations complete (out of 30,000): 19000
Iterations complete (out of 30,000): 20000
Iterations co

In [ ]:
# FILE CHECKING

# checking that your saving sensible qa pairs
for i in range(15):
    print(f"Example {i+1}:")
    print("Question:", num_exs[i]['question'])
    print("Context:", num_exs[i]['context'])
    print("Answer:", num_exs[i]['answers'])
    print("\n")

Example 1:
Question: Which election was the Contract with America unveiled closest to?
Context: Second, Democrats have always elevated their minority floor leader to the speakership upon reclaiming majority status. Republicans have not always followed this leadership succession pattern. In 1919, for instance, Republicans bypassed James R. Mann, R-IL, who had been minority leader for eight years, and elected Frederick Gillett, R-MA, to be Speaker. Mann "had angered many Republicans by objecting to their private bills on the floor;" also he was a protégé of autocratic Speaker Joseph Cannon, R-IL (1903–1911), and many Members "suspected that he would try to re-centralize power in his hands if elected Speaker." More recently, although Robert H. Michel was the Minority Leader in 1994 when the Republicans regained control of the House in the 1994 midterm elections, he had already announced his retirement and had little or no involvement in the campaign, including the Contract with America wh

In [ ]:
# SAVING

# saving file as a json to add to training
import json

with open('/content/num_exs.json', 'w') as json_file:
    json.dump(num_exs, json_file, indent=2)

"num_exs_dict = num_exs.to_dict()\nnum_exs_json = json.dumps(num_exs_dict, indent=2)\n\nwith open('/content/num_exs.json', 'w') as json_file:\n    json.dump(num_exs_dict, json_file, indent=2)"

**Synonym Examples From "Adversarial QA"**

In [ ]:
# EXTRACTING SHUFFLED SENTENCES TO REPLACE W SYNONYMS
import pandas as pd

syn_exs = []
ex_goal = 0

adversarial_qa_dataset_shuffled = adversarial_qa_dataset.shuffle(seed=505)

# iterate through until 500 exs or finish dataset
for i in range(len(adversarial_qa_dataset)):
    # extracting answers and splitting into words
    answer = adversarial_qa_dataset['answers'][i]
    answer_words = answer['text'][0].split()
    # keep answers longer than 5 words
    if len(answer_words) > 5:
      # adding answers to a list for json conversion
      example = {
      'question': adversarial_qa_dataset['question'][i],
      'context': adversarial_qa_dataset['context'][i],
      'answers': adversarial_qa_dataset['answers'][i],
    }
      syn_exs.append(example)

      # increment ex_goal count
      ex_goal += 1

    # once you reach 500, break
    if ex_goal == 500:
      break

# CONVERT TO DF AND SAVE AS XLSX

df = pd.DataFrame(syn_exs)

# split up the first data columnn into its keys
df_normalized = pd.json_normalize(df['answers'])
# concatenate original df with the new split df
df_result = pd.concat([df, df_normalized], axis=1)
# drop the original combined data column
df_result = df_result.drop('answers', axis=1)

# convert to excel
df_result.to_excel('syn_exs.xlsx', index=False, engine='openpyxl')

In [ ]:
# FILE CHECKING

# checking that your saving sensible qa pairs
for i in range(15):
    print(f"Example {i+1}:")
    print("Question:", syn_exs[i]['question'])
    print("Context:", syn_exs[i]['context'])
    print("Answer:", syn_exs[i]['answers'])
    print("\n")

Example 1:
Question: What is an advantage of testing brain function in animals?
Context: Another approach to brain function is to examine the consequences of damage to specific brain areas. Even though it is protected by the skull and meninges, surrounded by cerebrospinal fluid, and isolated from the bloodstream by the blood–brain barrier, the delicate nature of the brain makes it vulnerable to numerous diseases and several types of damage. In humans, the effects of strokes and other types of brain damage have been a key source of information about brain function. Because there is no ability to experimentally control the nature of the damage, however, this information is often difficult to interpret. In animal studies, most commonly involving rats, it is possible to use electrodes or locally injected chemicals to produce precise patterns of damage and then examine the consequences for behavior.
Answer: {'text': ['it is possible to use electrodes or locally injected chemicals to produce

In [ ]:
# ADDING EDITED EXAMPLES BACK IN
import pandas as pd

# importing edited examples
syn_exs_edited_file = pd.read_excel("/content/syn_exs_edited.xlsx")

#for i in range(5):
  #print(syn_exs_edited_file['new_text'][i])

syn_exs_edited = []
ex_goal = 0

adversarial_qa_dataset_shuffled = adversarial_qa_dataset.shuffle(seed=505)

# iterate through until 500 exs or finish dataset
for i in range(len(adversarial_qa_dataset)):

    # extracting answers and splitting into words
    answer = adversarial_qa_dataset['answers'][i]
    answer_words = answer['text'][0].split()

    # replace answers longer than 5 words with edited answers
    if len(answer_words) > 5:
      # print(adversarial_qa_dataset['answers'][i]['text'])
      # print(syn_exs_edited_file['new_text'][ex_goal])

      # adding answers to a list for json conversion
      example = {
      'question': adversarial_qa_dataset['question'][i],
      'context': adversarial_qa_dataset['context'][i],
      'answers': {
          'text': syn_exs_edited_file['new_text'][ex_goal],
          'answer_start': adversarial_qa_dataset['answers'][i]['answer_start']
      }
  }
      syn_exs_edited.append(example)

      # increment ex_goal count
      ex_goal += 1

    # once you reach 500, break
    if ex_goal == 500:
      break

In [ ]:
# FILE CHECKING

# checking that your saving sensible qa pairs
for i in range(15):
    print(f"Example {i+1}:")
    print("Question:", syn_exs_edited[i]['question'])
    print("Context:", syn_exs_edited[i]['context'])
    print("Answer:", syn_exs_edited[i]['answers'])
    print("\n")

Example 1:
Question: What is an advantage of testing brain function in animals?
Context: Another approach to brain function is to examine the consequences of damage to specific brain areas. Even though it is protected by the skull and meninges, surrounded by cerebrospinal fluid, and isolated from the bloodstream by the blood–brain barrier, the delicate nature of the brain makes it vulnerable to numerous diseases and several types of damage. In humans, the effects of strokes and other types of brain damage have been a key source of information about brain function. Because there is no ability to experimentally control the nature of the damage, however, this information is often difficult to interpret. In animal studies, most commonly involving rats, it is possible to use electrodes or locally injected chemicals to produce precise patterns of damage and then examine the consequences for behavior.
Answer: {'text': "['it is possible to use electrodes or directly injected chemicals to produ

In [ ]:
# SAVING

# saving file as a json to add to training
import json

with open('/content/syn_exs_edited.json', 'w') as json_file:
    json.dump(syn_exs_edited, json_file, indent=2)

In [ ]:
# CHECKING FILE IS PROPERLY STORED
with open("/content/syn_exs_edited.json", 'r') as file:
    data = json.load(file)

print(len(data))

500


In [ ]:
# FIXING THE TEXT FIELD

import json
import ast

json_file_path = "/content/syn_exs_edited.json"

with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

for example in data:
    # extracting the literal from the string aka making it a normal list
    example["answers"]["text"] = ast.literal_eval(example["answers"]["text"])

# write back to same location
with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file, indent=2)

In [ ]:
# PRELIM ANALYSIS

# EXTRACTING NUMERIC ANSWERS

one_ex = []

# iterate through set number of examples and take out examples with number answers
for i in range(1):
    answer = adversarial_qa_dataset['answers'][i]
    answer_words = answer['text'][0].split()

    example = {
    'question': adversarial_qa_dataset['question'][i],
    'context': adversarial_qa_dataset['context'][i],
    'answers': adversarial_qa_dataset['answers'][i],
  }
    one_ex.append(example)

# SAVING

# saving file as a json to add to training
import json

with open('/content/one_ex.json', 'w') as json_file:
    json.dump(one_ex, json_file, indent=2)

**Trained Adv Results**

In [2]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00


In [8]:
import json
import random
import pandas as pd
from datasets import load_metric

# Load the SQuAD metric
squad_metric = load_metric('squad')

# Desired size of the results list
num_examples = 500

# Load all lines into a list
with open('/content/eval_predictions_11.jsonl', 'r') as file:
    lines = file.readlines()

# Parse JSON data
data = [json.loads(line) for line in lines]

# Shuffle the data
random.shuffle(data)

# List to store the results
results_list = []

# Process each index in the sample
for index in range(len(data)):
    # Define the prediction and reference
    prediction = {"id": data[index]["id"], "prediction_text": data[index]["predicted_answer"]}
    reference = {"id": data[index]["id"], "answers": data[index]["answers"]}

    # Compute the F1 score
    results = squad_metric.compute(predictions=[prediction], references=[reference])

    # If the F1 score is lower than 0.5, add the result to the list
    if results["f1"] < 0.5:
        result_entry = {
            "id": data[index]["id"],
            "title": data[index]["title"],
            "context": data[index]["context"],
            "question": data[index]["question"],
            "answers_text": data[index]["answers"]["text"],
            "answers_start": data[index]["answers"]["answer_start"],
            "predicted_answer": data[index]["predicted_answer"],
            "f1": results["f1"]
        }
        results_list.append(result_entry)

        if len(results_list) == 500:
          break

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results_list)

# Save the DataFrame to an Excel file
results_df.to_excel('results_new.xlsx', index=False)